In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#导入频率文件
cityfre=pd.read_csv('cityfre.csv')
devicefre=pd.read_csv('devicefre.csv')
ipfre=pd.read_csv('ipfre.csv')

In [ ]:
#导入训练登录文件并且添加频率
login=pd.read_csv('t_login.csv')
login['time']=pd.to_datetime(login['time'])
login=login.sort_values(by='id')
login=pd.merge(login,cityfre,on=['id','city'])
login=pd.merge(login,devicefre,on=['id','device'])
login=pd.merge(login,ipfre,on=['id','ip'])

In [ ]:
#导入训练交易文件
trade=pd.read_csv('t_trade.csv')
trade['time']=pd.to_datetime(trade['time'])
trade=trade.sort_values(by='id').reset_index(drop=True)
#初始化为0
trade['city_changed']=0#4
trade['device_changed']=0
trade['ip_changed']=0
trade['result_times']=0
trade['login_times']=0
trade['trade_times']=0
trade['city_times']=0
trade['device_times']=0
trade['ip_times']=0
trade['result_1']=0
trade['result_2']=0
trade['result_3']=0
trade['city_fre']=0#16
trade['device_fre']=0
trade['ip_fre']=0
trade['timelong']=0
trade['city']=0
trade['device']=0
trade['ip']=0

In [ ]:
#导入测试训练文件并且添加频率
login_test=pd.read_csv('t_login_test.csv')
login_test['time']=pd.to_datetime(login_test['time'])
login_test=login_test.sort_values(by='id').reset_index(drop=True)
login_test=pd.merge(login_test,cityfre,on=['id','city'])
login_test=pd.merge(login_test,devicefre,on=['id','device'])
login_test=pd.merge(login_test,ipfre,on=['id','ip'])

In [ ]:
#导入测试交易文件
trade_test=pd.read_csv('t_trade_test.csv')
trade_test['time']=pd.to_datetime(trade_test['time'])
trade_test=trade_test.sort_values(by='id').reset_index(drop=True)
#初始化为0
trade_test['city_changed']=0#3
trade_test['device_changed']=0
trade_test['ip_changed']=0
trade_test['result_times']=0
trade_test['login_times']=0
trade_test['trade_times']=0
trade_test['city_times']=0
trade_test['device_times']=0
trade_test['ip_times']=0
trade_test['result_1']=0
trade_test['result_2']=0
trade_test['result_3']=0
trade_test['city_fre']=0#15
trade_test['device_fre']=0
trade_test['ip_fre']=0
trade_test['timelong']=0
trade_test['city']=0
trade_test['device']=0
trade_test['ip']=0

step1：提取交易前1h内城市发生变化的，暂定为风险

In [ ]:
for j in range(trade_test.shape[0]):
    per_id=login_test[login_test.id==trade_test.iloc[j,2]].reset_index(drop=True)
    if per_id.empty==True:
        continue
    list_city=[]
    for i in range(per_id.shape[0]):
        timecha=trade_test.iloc[j,1]-per_id.time[i]
        if timecha.days==0 and timecha.seconds<=3600:
            list_city.append(per_id.city[i])
    city_only=list(set(list_city))
    trade_test.iloc[j,9]=len(city_only)
    if len(city_only)>1:
        trade_test.iloc[j,3]=1

step2：提取1h内设备发生变化的

In [ ]:
for j in range(trade_test.shape[0]):
    per_id=login_test[login_test.id==trade_test.iloc[j,2]].reset_index(drop=True)
    if per_id.empty==True:
        continue
    list_device=[]
    for i in range(per_id.shape[0]):
        timecha=trade_test.iloc[j,1]-per_id.time[i]
        if timecha.days==0 and timecha.seconds<=3600:
            list_device.append(per_id.device[i])
    device_only=list(set(list_device))
    trade_test.iloc[j,10]=len(device_only)
    if len(device_only)>1:
        trade_test.iloc[j,4]=1

step3：提取1h内ip发生变化的

In [ ]:
for j in range(trade_test.shape[0]):
    per_id=login_test[login_test.id==trade_test.iloc[j,2]].reset_index(drop=True)
    if per_id.empty==True:
        continue
    list_ip=[]
    for i in range(per_id.shape[0]):
        timecha=trade_test.iloc[j,1]-per_id.time[i]
        if timecha.days==0 and timecha.seconds<=3600:
            list_ip.append(per_id.ip[i])
    ip_only=list(set(list_ip))
    trade_test.iloc[j,11]=len(ip_only)
    if len(ip_only)>1:
        trade_test.iloc[j,5]=1

step4:提取1小时内result结果不成功的次数，以及倒数第一次、第二次、第三次是否登录错误，添加timelong

In [ ]:
for j in range(trade_test.shape[0]):
    per_id=login_test[login_test.id==trade_test.iloc[j,2]].reset_index(drop=True)
    if per_id.empty==True:
        continue
    list_result=[]
    per_id['timetheta']=0
    for i in range(per_id.shape[0]):
        timecha=trade_test.iloc[j,1]-per_id.time[i]
        if timecha.days==0 and timecha.seconds<=3600:
            list_result.append(per_id.result[i])
            per_id.loc[i,'timetheta']=timecha.seconds
    times=0
    for m in list_result:
        if m!=1:
            times=times+1
    trade_test.iloc[j,6]=times
    per_id=per_id.sort_values(by='timetheta')
    trade_test.iloc[j,18]=per_id.iloc[0,1]
    y=min(3,per_id.shape[0])
    for n in range(y):
        trade_test.iloc[j,(12+n)]=per_id.iloc[n,6]

step5:提取前后1h的登录次数

In [ ]:
for j in range(trade_test.shape[0]):
    per_id=login_test[login_test.id==trade_test.iloc[j,2]].reset_index(drop=True)
    if per_id.empty==True:
        continue
    times=0
    for i in range(per_id.shape[0]):
        if trade_test.iloc[j,1]<per_id.time[i]:
            timecha=per_id.time[i]-trade_test.iloc[j,1]
        else:
            timecha=trade_test.iloc[j,1]-per_id.time[i]
        if timecha.days==0 and timecha.seconds<=3600:
            times=times+1
    trade_test.iloc[j,7]=times

step6:提取1h的交易次数

In [ ]:
for j in range(trade_test.shape[0]):
    per_id=trade_test[trade_test.id==trade_test.iloc[j,2]].reset_index(drop=True)
    if per_id.empty==True:
        continue
    times=0
    for i in range(per_id.shape[0]):
        if trade_test.iloc[j,1]<per_id.time[i]:
            timecha=per_id.time[i]-trade_test.iloc[j,1]
        else:
            timecha=trade_test.iloc[j,1]-per_id.time[i]
        if timecha.days==0 and timecha.seconds<=3600:
            times=times+1
    trade_test.iloc[j,8]=times

step7:凌晨交易

In [ ]:
for j in range(trade_test.shape[0]):
    hour=trade_test.iloc[j,1].hour
    if hour>=2 and hour<=5:
        trade_test.iloc[j,9]=1        

step8：构造城市、设备、ip、城市频率、设备频率、ip频率特征

In [ ]:
#训练集
for j in range(trade.shape[0]):
    per_id=login[login.id==trade.iloc[j,2]].reset_index(drop=True)
    if per_id.empty==True:
        continue
    per_id['timetheta']=0
    m=0
    for i in range(per_id.shape[0]):
        timecha=trade.iloc[j,1]-per_id.time[i]
        if timecha.days==0 and timecha.seconds<=3600:
            per_id.loc[i,'timetheta']=timecha.seconds
            m=m+1
    if m==0:
        continue
    per_id=per_id.sort_values(by='timetheta')
    trade.iloc[j,16:19]=list(per_id.iloc[0,13:16])
    trade.iloc[j,20]=per_id.iloc[0,5]
    trade.iloc[j,21]=per_id.iloc[0,2]
    trade.iloc[j,22]=per_id.iloc[0,4]

In [ ]:
#测试集
for j in range(trade_test.shape[0]):
    per_id=login_test[login_test.id==trade_test.iloc[j,2]].reset_index(drop=True)
    if per_id.empty==True:
        continue
    per_id['timetheta']=0
    m=0
    for i in range(per_id.shape[0]):
        timecha=trade_test.iloc[j,1]-per_id.time[i]
        if timecha.days==0 and timecha.seconds<=3600:
            per_id.loc[i,'timetheta']=timecha.seconds
            m=m+1
    if m==0:
        continue
    per_id=per_id.sort_values(by='timetheta')
    trade_test.iloc[j,15:18]=list(per_id.iloc[0,13:16])
    trade_test.iloc[j,19]=per_id.iloc[0,5]
    trade_test.iloc[j,20]=per_id.iloc[0,2]
    trade_test.iloc[j,21]=per_id.iloc[0,4]

其他特征：同个ip短时间内登录多个id，上了90

In [ ]:
trade_test.to_csv('test_feature.csv')